# Packages

In [2]:
# Visualisation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from pyvis.network import Network
import shap


# Data analysis / Data processing
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = "{:,.2f}".format
from datetime import time, timedelta, datetime
import numpy as np
import networkx as nx
from collections import defaultdict
import ast
from ydata_profiling import ProfileReport
import re
from typing import Set, List, Dict
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Maths & Stats
import math 
import scipy.stats as st
from scipy import stats
from scipy.stats import norm
import statsmodels.stats.weightstats as ws
from statsmodels.stats.proportion import test_proportions_2indep
import AB_library
import random

# System library
import os
import ipywidgets
import warnings
import pandas_gbq
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
tqdm.pandas()
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_format='retina'
# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
import openpyxl

# Data connection
from google.cloud import bigquery
bigquery_client = bigquery.Client(project='analytics-dev-333113')


# Useful functions
def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {date} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    counter = 0

    for date in daterange:
        counter+=1
        print(f"{counter}) Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline"'), 
        raw=True
    )

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:

    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                            )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                            )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                            )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                            )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                            )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                            )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                            )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                        )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                        )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                    )

        print('DataFrame is written to Excel File successfully.') 

# Research

### Preparing the data

In [ ]:
# df_sample = pd.read_csv('analytics_dev_333113ty_score_total_for_ml.csv')

df_sample = pd.read_csv('/Users/renatyunison/Desktop/VSC scripts/Working/new_data_scoring.csv')

df_sample.head()

In [ ]:
def get_device_category(model: str) -> str:

    device_price_map = {
    'pro max': 'premium', 'ultra': 'premium', 'galaxy z': 'premium',
    'galaxy s': 'premium', 'fold': 'premium', 'flip': 'premium',
    'pixel': 'premium', 'pro': 'premium', 'iphone': 'premium',
    'rog': 'premium', 'sony': 'premium', 
    
    'oneplus': 'mid-range', 'honor': 'mid-range', 'huawei': 'mid-range',
    'galaxy a': 'mid-range', 'samsung': 'mid-range', 'pixel a': 'mid-range',
    'nord': 'mid-range', 'realme': 'mid-range', 'oppo': 'mid-range',
    'xiaomi': 'mid-range', 'motorola': 'mid-range', 'vivo': 'mid-range',
    'iqoo': 'mid-range', 'lge': 'mid-range', 'nothing': 'mid-range',
    'tcl': 'mid-range', 'nubia': 'mid-range', 'asus': 'mid-range',
    
    'sm-a': 'budget', 'moto g': 'budget', 'lite': 'budget',
    'redmi': 'budget', 'poco': 'budget', 'infinix': 'budget',
    'tecno': 'budget', 'lenovo': 'budget', 'nokia': 'budget',
    'itel': 'budget', 'lava': 'budget', 'zte': 'budget', 'op': 'budget',
    'ulefone': 'budget', 'sparx': 'budget', 'hisense': 'budget',
    'blackview': 'budget', 'cubot': 'budget', 'doogee': 'budget',
    'benco': 'budget', 'vgo_tel': 'budget', 'hmd': 'budget',
    'villaon': 'budget', 
    'vios': 'budget', 
    'marcel': 'budget', 
    'cg_mobiles': 'budget', 
    'ulesson': 'budget', 
    'mobice': 'budget', 
    'mirage': 'budget' 
}

    if not isinstance(model, str):
        return 'unknown'

    cleaned_model = model.lower().strip()
    cleaned_model = re.sub(r'[^\w\s]', '', cleaned_model)
    cleaned_model = re.sub(r'\s+', ' ', cleaned_model).strip()

    for keyword, category in device_price_map.items():
        if keyword in cleaned_model:
            return category
    
    return 'unknown' 


df_sample['device_price_category'] = df_sample['devicemodel'].apply(get_device_category)

df_sample.head()

In [ ]:
def create_name_suspicious_flag(df: pd.DataFrame) -> pd.DataFrame:

    import unicodedata

    common_names_whitelist = {
        'michael', 'john', 'anna', 'mary', 'david', 'chris', 'james', 'anna',
        'robert', 'william', 'joseph', 'charles', 'thomas', 'daniel', 'matthew',
        'elizabeth', 'sandra', 'barbara', 'patricia', 'jennifer', 'maria',
        'hector', 'adriana', 'rodrigo', 'italo', 'karen', 'julia', 'fabio', 'antonio',
        'thaylla', 'rania', 'lwazi', 'miriam', 'ronaldo', 'costanza', 'fernando',
        'vanessa', 'nigel', 'rudy', 'dimitri', 'sharon', 'mavuzo', 'ana', 'danny',
        'paola', 'david', 'camila', 'alexis', 'alejandra', 'dante', 'juanita',
        'wilmer', 'william', 'fernanda', 'leon', 'jeremy', 'atif', 'judith',
        'thiago', 'michel', 'didier', 'isaias', 'melissa', 'nayara', 'kaitlin',
        'karla', 'anthony', 'gerson', 'daniel', 'alfred', 'ailton', 'victoria',
        'mirzhan', 'миржан' # Добавлены имена из вашего списка
    }

    suspicious_ngrams = {
        'vv', 'ww', 'qq', 'jj', 'kk', 'uu', 'oo', 'yy', 'zz',
        'qwe', 'asd', 'zxc', 'rty', 'fgh', 'vbn', 'tyu', 'ghj', 'bnm', 'dfg', 'jkl', 'uio', 'iop', 'klj',
        'j.d', 'j.r', 'j d', 'j r', 'k d', 'l d', 'l r', 'r d', 'jd', 'jr', 'kl', 'gh', 'kk', 'qq',
        '123', '234', '345', '456', '567', '678', '789', '890', '000',
        'abc', 'xyz', 'mnp', 'qaz', 'wsx', 'edc',
        'b1', 'c1', 'd1', 'e1', 'f1',
        '1a', '2a', '3a', '4a', '5a',
    }

    def has_repeated_chars(s: str) -> bool:
        """
        Проверяет, содержит ли строка повторяющийся символ 3 или более раз подряд.
        """
        return bool(re.search(r'(.)\1\1', s))
    
    def normalize_name(s: str) -> str:
        """Конвертирует строку в ASCII-представление, убирая акценты и диакритические знаки."""
        return unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('utf-8')

    def check_name_adequacy(name: str) -> int:
        name_str = str(name).strip()
        name_lower = name_str.lower()
        
        # ПРАВИЛО 0: Проверка по "белому списку" (перед нормализацией)
        if name_lower in common_names_whitelist:
            return 0

        # ПРАВИЛО 1 (НОВОЕ): Нормализация имени
        # Применяем нормализацию к имени перед дальнейшими проверками
        normalized_name_str = normalize_name(name_str)
        normalized_name_lower = normalized_name_str.lower()
        
        # ПРАВИЛО 2: Проверка на повторяющиеся символы
        if has_repeated_chars(normalized_name_lower):
            return 1

        # Правило 3: Имя слишком короткое
        name_no_space = normalized_name_lower.replace(' ', '')
        if len(name_no_space) < 3:
            return 1
            
        # Правило 4: Имя состоит из одной или двух букв, которые являются заглавными
        if len(name_no_space) <= 2 and normalized_name_str.isupper():
            return 1

        # Правило 5: Наличие необычных символов или цифр
        # Теперь это правило будет проверять только те символы, которые остались после нормализации
        if bool(re.search(r'[^a-zA-Zа-яА-Я\s]', normalized_name_str)):
            return 1
            
        # Правило 6: Имя написано полностью в нижнем регистре
        if normalized_name_str.islower():
            return 1

        # Правило 7: Наличие подозрительных n-грамм
        for i in range(len(normalized_name_lower) - 1):
            if normalized_name_lower[i:i+2] in suspicious_ngrams:
                return 1
        for i in range(len(normalized_name_lower) - 2):
            if normalized_name_lower[i:i+3] in suspicious_ngrams:
                return 1

        return 0

    df['name_suspicious_flag'] = df['firstname'].apply(check_name_adequacy)
    return df


df_sample_for_ml = create_name_suspicious_flag(df_sample)

df_sample_for_ml.head()

In [ ]:
df_sample_for_ml['price_order_usd'] = pd.to_numeric(df_sample_for_ml['price_order_usd'], errors='coerce').fillna(0)
df_sample_for_ml['distance_kilometers'] = pd.to_numeric(df_sample_for_ml['distance_kilometers'], errors='coerce').fillna(0)

df_sample_for_ml['price_per_meter'] = df_sample_for_ml['price_order_usd'] / (df_sample_for_ml['distance_kilometers'] + 0.001)

df_sample_for_ml.head()

In [ ]:
df_sample_for_ml.to_csv('df_sample_for_ml.csv', index=False)

### Gradient boosting

In [ ]:
df_sample_for_ml = pd.read_csv("/Users/renatyunison/Desktop/VSC scripts/Working/df_sample_for_ml.csv")


user_ids = df_sample_for_ml['user_id']
incident_level = df_sample_for_ml['incident_level']
incident_type = df_sample_for_ml['incident_type']
city_id = df_sample_for_ml['city_id']

print("Данные загружены. Размер:", df_sample_for_ml.shape)

print("Предобработка данных...")

TARGET = 'incident_flag'

X = df_sample_for_ml[
    ['user_id', 'incident_level', 'incident_type', 'lifetime', 'appropriate_avatar_flag', 
    'os_version', 'nighttime_flag', 'distance_meters', 
    'price_order_usd', 'total_orders', 'total_cancel_orders', 'total_cancel_accepted_orders', 'total_rides', 'cancel_orders_1d_window', 
    'cancel_accepted_orders_1d_window', 'orders_1d_window', 'rides_1d_window', 'total_device_change', 
    'total_nickname_change', 'total_avatar_change', 'device_change_1d_window', 'nickname_change_1d_window', 
    'avatar_change_1d_window', 'device_change_6h_window', 'nickname_change_6h_window', 'avatar_change_6h_window', 
    'device_change_7d_window', 'nickname_change_7d_window', 'avatar_change_7d_window',
    'day_diff_before_order', 'first_day_order', 'customer_rating_1w_window', 'customer_rating_1m_window', 'customer_rating_total', 
    'bans_before_order', 'appeals_2d_window', 'appeals_1w_window', 'appeals_2w_window', 'appeals_all_time', 
    'reviews_2d_window', 'reviews_1w_window', 'reviews_2w_window', 'reviews_all_time', 'geo_permission_2d_window', 
    'device_price_category', 'price_per_meter', 'name_suspicious_flag', 'incident_flag', 'complaints_in_feed', 'appeals_feed_1m', 'appeals_feed_7w']

       ].drop(columns=[TARGET, 'user_id', 'incident_level', 'incident_type'])
y = df_sample_for_ml[TARGET]

categorical_features: List[str] = [
    col for col in X.columns if X[col].dtype == 'object'
]
for col in categorical_features:
    X[col] = X[col].astype('category')

X['cancel_ratio'] = X['total_cancel_orders'] / X['total_orders']
X['cancel_ratio'] = X['cancel_ratio'].fillna(0)

X['cancel_ratio_1d'] = X['cancel_accepted_orders_1d_window'] / X['orders_1d_window']
X['cancel_ratio_1d'] = X['cancel_ratio_1d'].fillna(0)

numerical_features = X.select_dtypes(include=np.number).columns.tolist()
for col in numerical_features:
    if X[col].isnull().any():
        X[col] = X[col].fillna(0)

print("Предобработка завершена.")

print("Разделение данных и обучение модели...")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'n_estimators': 150,
    'learning_rate': 0.05,
    'num_leaves': 35,
    'max_depth': -1,
    'random_state': 42,
    'n_jobs': -1,
    'class_weight': 'balanced',
    'verbose': -1
}

model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train,
          categorical_feature=categorical_features)

print("Обучение модели завершено.")

y_pred_proba = model.predict_proba(X_test)[:, 1]

# Сохраняем исходные колонки тестового набора для фильтрации
df_test = df_sample_for_ml.iloc[y_test.index]
df_test['y_test'] = y_test
df_test['y_pred_proba'] = y_pred_proba
df_test = pd.merge(df_test, incident_level, left_index=True, right_index=True)
df_test = pd.merge(df_test, city_id, left_index=True, right_index=True)

df_test.drop(columns='incident_level_y', inplace=True)
df_test.rename(columns={'incident_level_x':'incident_level'}, inplace=True)
df_test.drop(columns='city_id_y', inplace=True)
df_test.rename(columns={'city_id_x':'city_id'}, inplace=True)

mapping_df = df_test.dropna(subset=['incident_level'])

if not mapping_df.groupby('incident_type')['incident_level'].nunique().le(1).all():
    type_to_level_mapping = mapping_df.groupby('incident_type')['incident_level'].agg(lambda x: x.mode()[0]).to_dict()
else:
    type_to_level_mapping = mapping_df.set_index('incident_type')['incident_level'].to_dict()

df_test['incident_level'] = df_test['incident_level'].fillna(
    df_test['incident_type'].map(type_to_level_mapping)
)

df_test.to_csv('df_test.csv', index=True)

y_pred_proba_full = model.predict_proba(X)[:, 1]

final_df = df_sample_for_ml.copy()
final_df['user_id'] = user_ids
final_df['incident_level'] = incident_level
final_df['incident_type'] = incident_type
final_df['predicted_probability'] = y_pred_proba_full
final_df['predicted_class'] = (final_df['predicted_probability'] >= 0.86).astype(int) # Порог 0.5 для примера

final_df.to_csv('final_df.csv')


print("Расчет метрик для разных порогов...")
metrics_df = pd.DataFrame(columns=[
    'Threshold',
    'True Positives (TP)', 'False Positives (FP)',
    'False Negatives (FN)', 'True Negatives (TN)',
    'Recall',
    'Precision',
    'Accuracy', 'F1-Score', 'FPR'
])
thresholds = np.arange(0.01, 1.0, 0.01)
for threshold in thresholds:
    y_pred_thresholded = (y_pred_proba >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thresholded).ravel()
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    metrics_df.loc[len(metrics_df)] = [
        threshold, tp, fp, fn, tn,
        recall, precision, accuracy, f1, fpr
    ]
metrics_df['Recall'] = (metrics_df['Recall'] * 100).round(3)
metrics_df['Precision'] = (metrics_df['Precision'] * 100).round(3)
metrics_df['Accuracy'] = (metrics_df['Accuracy'] * 100).round(3)
metrics_df['F1-Score'] = (metrics_df['F1-Score'] * 100).round(3)
metrics_df['FPR'] = (metrics_df['FPR'] * 100).round(3)

print("\n" + "="*50)
print("Финальная таблица метрик:")
print("="*50)
print(metrics_df)

print("\n" + "="*50)
print("Анализ важности признаков")
print("="*50)

# Получаем важность признаков
feature_importances = model.feature_importances_

# Создаем DataFrame для важности признаков
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Сортируем по важности
importance_df = importance_df.sort_values(by='Importance', ascending=False)



In [3]:
df = pd.read_csv('/Users/renatyunison/Desktop/VSC scripts/Working/final_df.csv')

df.head()

,Unnamed: 0,user_id,lifetime,avatar_flag,appropriate_avatar_flag,city_id,devicemodel,os_version,firstname,lastname,phone,registration_date,nighttime_flag,order_id,order_uuid,order_timestamp,incident_flag,incident_level,incident_type,distance_meters,distance_kilometers,price_order_usd,avg_price_order_usd,ratio_price_order_usd,total_orders,total_cancel_orders,total_cancel_accepted_orders,total_rides,cancel_orders_1d_window,cancel_accepted_orders_1d_window,orders_1d_window,rides_1d_window,total_device_change,total_nickname_change,total_avatar_change,device_change_1d_window,nickname_change_1d_window,avatar_change_1d_window,device_change_6h_window,nickname_change_6h_window,avatar_change_6h_window,device_change_7d_window,nickname_change_7d_window,avatar_change_7d_window,device,liveness,device_liveness,last_ride_date,day_diff_before_order,first_day_order,customer_rating_1w_window,customer_rating_1m_window,customer_rating_total,bans_before_order,appeals_2d_window,appeals_1w_window,appeals_2w_window,appeals_all_time,reviews_2d_window,reviews_1w_window,reviews_2w_window,reviews_all_time,geo_permission_2d_window,complaints_in_feed,appeals_feed_1m,appeals_feed_7w,device_price_category,name_suspicious_flag,price_per_meter,predicted_probability,predicted_class
0,0,289646646,139,0,0,5640,V2333 vivo V2333,Android,ridwan,NaN,6282349886014,2025-04-09,0,"3,874,366,842,494,306,304.00",0196cce4-1873-7989-8776-12d8f31d879e,2025-05-14 11:43:42.000000,0,NaN,NaN,994.17,0.99,0.85,3.17,0.27,14.00,6.00,6.00,7.00,1,1,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2025-04-17,27,0,5.00,5.00,5.00,0,0,0,0,0,0,0,0,0,unknown,0,0,0,mid-range,1,0.85,0.36,0
1,1,290241495,136,1,0,4532,a54x samsung SM-A546E,Android,Ana Luiza,NaN,5562991466332,2025-04-12,1,"2,343,484,012,339,849,728.00",0197fcc1-4900-800b-c05a-f89388b5e54a,2025-07-12 00:50:15.000000,0,NaN,NaN,"3,141.22",3.14,2.88,3.37,0.85,22.00,12.00,1.00,9.00,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2025-07-11,1,0,3.81,3.81,4.70,0,0,0,0,0,0,0,0,0,enable,0,0,0,mid-range,0,0.92,0.62,0
2,2,295896279,101,1,0,6079,a15x samsung SM-A156M,Android,Miguel Ángel Aníbal,NaN,50254434430,2025-05-17,1,"6,807,366,403,132,298,240.00",0197bb83-4fcd-8017-c040-d4766e5f44a7,2025-06-29 05:47:14.000000,0,NaN,NaN,"1,488.64",1.49,2.60,2.58,1.01,7.00,1.00,0.00,6.00,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2025-06-16,13,0,4.77,4.73,4.67,0,0,0,0,0,0,0,0,0,unknown,0,0,0,mid-range,0,1.75,0.14,0
3,3,275248105,230,0,0,10155,blue Redmi 23129RN51X,Android,صلاح,NaN,201151905046,2025-01-08,1,"3,738,610,074,304,463,872.00",01955e9c-ac80-7926-804d-a44b938a1120,2025-03-04 02:44:40.000000,0,NaN,NaN,"5,632.93",5.63,1.96,2.26,0.87,17.00,11.00,4.00,4.00,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2025-03-01,3,0,1.00,1.00,1.14,1,0,0,0,0,0,0,0,0,unknown,0,0,0,budget,1,0.35,0.65,0
4,4,292936861,119,0,0,840,iPhone 16 Pro,IOS,Anne,NaN,639062671417,2025-04-29,1,"1,344,414,298,766,548,224.00",019681c3-b84b-7053-a4ef-b71d69715f4e,2025-04-29 21:36:49.000000,0,NaN,NaN,"53,539.52",53.54,26.89,5.33,5.05,2.00,2.00,0.00,0.00,1,0,1,0,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,NaN,0,1,4.80,4.80,4.80,0,0,0,0,0,0,0,0,0,unknown,0,0,0,premium,0,0.50,0.01,0


In [10]:
df['order_timestamp'] = pd.to_datetime(df['order_timestamp'])


pandas_gbq.to_gbq(df[['user_id', 'city_id', 'order_timestamp', 'predicted_probability', 'predicted_class']],
                destination_table='renat_yunisov.lgb_results_safety_scoring',
                project_id='analytics-dev-333113',
                if_exists='append')

100%|██████████| 1/1 [00:00<00:00, 10082.46it/s]


In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

roc_auc = auc(fpr, tpr)

print(f"Значение ROC-AUC на тестовой выборке: {roc_auc:.4f}")

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
metrics_df.sort_values('Precision', ascending=False).head(10)

In [ ]:
metrics_df[(metrics_df['Recall']>50)&(metrics_df['Recall']<70)]

In [ ]:
metrics_df.to_csv('metrics_df.csv')

In [ ]:
explainer = shap.TreeExplainer(model)
X_sample = X_test.sample(n=10000, random_state=42)
shap_values = explainer.shap_values(X_sample)
if isinstance(shap_values, list):
    if len(shap_values) > 1:
        shap_values_to_plot = shap_values[1]
    else:

        shap_values_to_plot = shap_values[0]
else:
    shap_values_to_plot = shap_values

shap.summary_plot(shap_values_to_plot, X_sample)

shap.summary_plot(shap_values_to_plot, X_sample, plot_type="bar")

In [ ]:

categorical_features = categorical_features + ['name_suspicious_flag']

for value in categorical_features:
    shap.dependence_plot(value, shap_values_to_plot, X_sample)


In [ ]:
importance_df

##### By Incident level

In [ ]:
model_res = pd.concat([y_test, df_test[['incident_level', 'y_pred_proba', 'city_id']]], axis=1)

levels = ['Red', 'Green', 'Yellow']

df_metric_res = pd.DataFrame()

for level in levels:
    print(f'Region: {level}')

    metrics_df_by_levels = pd.DataFrame(columns=[
        'Threshold',
        'True Positives (TP)', 'False Positives (FP)',
        'False Negatives (FN)', 'True Negatives (TN)',
        'Recall',
        'Precision',
        'Accuracy', 'F1-Score', 'FPR'
    ])


    thresholds = np.arange(0.01, 1.0, 0.01)
    for threshold in thresholds:
        
        y_pred_thresholded = (model_res[(model_res['incident_level'].isna())|(model_res['incident_level']==level)]['y_pred_proba'] >= threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(model_res[(model_res['incident_level'].isna())|(model_res['incident_level']==level)]['incident_flag'], y_pred_thresholded).ravel()
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        metrics_df_by_levels.loc[len(metrics_df_by_levels)] = [
            threshold, tp, fp, fn, tn,
            recall, precision, accuracy, f1, fpr
        ]
    metrics_df_by_levels['Recall'] = (metrics_df_by_levels['Recall'] * 100).round(3)
    metrics_df_by_levels['Precision'] = (metrics_df_by_levels['Precision'] * 100).round(3)
    metrics_df_by_levels['Accuracy'] = (metrics_df_by_levels['Accuracy'] * 100).round(3)
    metrics_df_by_levels['F1-Score'] = (metrics_df_by_levels['F1-Score'] * 100).round(3)
    metrics_df_by_levels['FPR'] = (metrics_df_by_levels['FPR'] * 100).round(3)
    metrics_df_by_levels['Level'] = level

    df_metric_res = pd.concat([df_metric_res, metrics_df_by_levels])

df_metric_res.head(10)

##### By Incident type

In [ ]:
model_res = pd.concat([y_test, df_test[['incident_type', 'y_pred_proba', 'city_id']]], axis=1)

df_metric_res = pd.DataFrame()

for level in tqdm(df_test[~df_test['incident_type'].isna()]['incident_type'].unique()):
    print(f'Region: {level}')

    metrics_df_by_levels = pd.DataFrame(columns=[
        'Threshold',
        'True Positives (TP)', 'False Positives (FP)',
        'False Negatives (FN)', 'True Negatives (TN)',
        'Recall',
        'Precision',
        'Accuracy', 'F1-Score', 'FPR'
    ])


    thresholds = np.arange(0.01, 1.0, 0.01)
    for threshold in thresholds:
        
        y_pred_thresholded = (model_res[(model_res['incident_type'].isna())|(model_res['incident_type']==level)]['y_pred_proba'] >= threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(model_res[(model_res['incident_type'].isna())|(model_res['incident_type']==level)]['incident_flag'], y_pred_thresholded).ravel()
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        metrics_df_by_levels.loc[len(metrics_df_by_levels)] = [
            threshold, tp, fp, fn, tn,
            recall, precision, accuracy, f1, fpr
        ]
    metrics_df_by_levels['Recall'] = (metrics_df_by_levels['Recall'] * 100).round(3)
    metrics_df_by_levels['Precision'] = (metrics_df_by_levels['Precision'] * 100).round(3)
    metrics_df_by_levels['Accuracy'] = (metrics_df_by_levels['Accuracy'] * 100).round(3)
    metrics_df_by_levels['F1-Score'] = (metrics_df_by_levels['F1-Score'] * 100).round(3)
    metrics_df_by_levels['FPR'] = (metrics_df_by_levels['FPR'] * 100).round(3)
    metrics_df_by_levels['Level'] = level

    df_metric_res = pd.concat([df_metric_res, metrics_df_by_levels])

df_metric_res.head(10)

In [ ]:
df_metric_res.to_csv('df_metric_res.csv', index=True)
model_res.to_csv('model_res.csv', index=True)


#### By cities

In [ ]:
df_test = pd.read_csv('df_test.csv')

df_test.head()

In [ ]:
df_cities = read_bq("""
SELECT city_id, country_id, country_name, macroregion_name
FROM indriver-e6e40.heap.vw_macroregion_mapping
""")

df_cities.head()

In [ ]:
df_test = df_test.merge(df_cities, left_on='city_id', right_on='city_id', how='inner')

df_test_by_region = df_test.rename(columns={'Unnamed: 0':'index'}).set_index('index')

df_test.shape

In [ ]:
model_res = pd.merge(y_test, df_test_by_region[['incident_level', 'y_pred_proba', 'country_name', 'macroregion_name']], left_index=True, right_index=True)


df_metric_res = pd.DataFrame()

for country in tqdm(model_res['macroregion_name'].unique()):
    print(f'Region: {country}')

    metrics_df_by_region = pd.DataFrame(columns=[
        'Threshold',
        'True Positives (TP)', 'False Positives (FP)',
        'False Negatives (FN)', 'True Negatives (TN)',
        'Recall',
        'Precision',
        'Accuracy', 'F1-Score', 'FPR'
    ])


    thresholds = np.arange(0.01, 1.0, 0.01)
    for threshold in thresholds:
        
        y_pred_thresholded = (model_res[(model_res['macroregion_name']==country)]['y_pred_proba'] >= threshold).astype(int)

        try:
            tn, fp, fn, tp = confusion_matrix(model_res[(model_res['macroregion_name']==country)]['incident_flag'], y_pred_thresholded).ravel()
        except ValueError:
            continue
        
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        metrics_df_by_region.loc[len(metrics_df_by_region)] = [
            threshold, tp, fp, fn, tn,
            recall, precision, accuracy, f1, fpr
        ]
    metrics_df_by_region['Recall'] = (metrics_df_by_region['Recall'] * 100).round(3)
    metrics_df_by_region['Precision'] = (metrics_df_by_region['Precision'] * 100).round(3)
    metrics_df_by_region['Accuracy'] = (metrics_df_by_region['Accuracy'] * 100).round(3)
    metrics_df_by_region['F1-Score'] = (metrics_df_by_region['F1-Score'] * 100).round(3)
    metrics_df_by_region['FPR'] = (metrics_df_by_region['FPR'] * 100).round(3)
    metrics_df_by_region['Level'] = country

    df_metric_res_reg = pd.concat([df_metric_res, metrics_df_by_region])

df_metric_res_reg.head(10)

In [ ]:
df_metric_res_reg.to_csv('df_metric_res_reg.csv', index=True)


In [ ]:
df_metric_res.sort_values('F1-Score', ascending=False).head(15)

### Log regression

In [ ]:
df_sample_for_ml[[
    'price_per_meter', 'price_order_usd', 'total_cancel_accepted_orders', 'customer_rating_1w_window', 'day_diff_before_order', 'lifetime',
    'total_rides', 'orders_1d_window', 'customer_rating_1m_window', 'bans_before_order', 'total_cancel_orders', 'total_orders', 'nighttime_flag', 'cancel_accepted_orders_1d_window'
    ] + ['user_id', 'incident_level', 'incident_type', 'city_id', 'complaints_in_feed', 'appeals_feed_1m', 'appeals_feed_7w', 'incident_flag']].head()


In [ ]:

df_sample_for_ml = pd.read_csv("/Users/renatyunison/Desktop/VSC scripts/Working/df_sample_for_ml.csv")

user_ids = df_sample_for_ml['user_id']
incident_level = df_sample_for_ml['incident_level']
incident_type = df_sample_for_ml['incident_type']
city_id = df_sample_for_ml['city_id']

df_full = df_sample_for_ml.copy()


sample_size_frac = 0.5


_, df_sampled = train_test_split(df_full, test_size=sample_size_frac, random_state=42, stratify=df_full['incident_flag'])

df = df_sampled.copy()


TARGET = 'incident_flag'

X = df[[
    'price_per_meter', 'price_order_usd', 'total_cancel_accepted_orders', 'customer_rating_1w_window', 'day_diff_before_order', 'lifetime',
    'total_rides', 'orders_1d_window', 'customer_rating_1m_window', 'bans_before_order', 'total_cancel_orders', 'total_orders', 'nighttime_flag', 'cancel_accepted_orders_1d_window'
    ] + ['user_id', 'incident_level', 'incident_type', 'city_id', 'complaints_in_feed', 'appeals_feed_1m', 'appeals_feed_7w', 'incident_flag']].drop(columns=[TARGET, 'user_id', 'incident_level', 'incident_type', 'city_id'])
y = df[TARGET]

numerical_features = X.select_dtypes(include=np.number).columns.tolist()
for col in numerical_features:
    if X[col].isnull().any():
        X[col] = X[col].fillna(0)

object_cols = X.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    X[col] = X[col].astype('category')

print("Предобработка данных для логистической регрессии...")

numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='category').columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'

print("Разделение данных и обучение модели...")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

logreg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42))
])

logreg_pipeline.fit(X_train, y_train)

print("Обучение модели завершено.")

y_pred_proba = logreg_pipeline.predict_proba(X_test)[:, 1]

print("Расчет метрик для разных порогов...")
metrics_df = pd.DataFrame(columns=[
    'Threshold',
    'True Positives (TP)', 'False Positives (FP)',
    'False Negatives (FN)', 'True Negatives (TN)',
    'Recall',
    'Precision',
    'Accuracy', 'F1-Score', 'FPR'
])

thresholds = np.arange(0.01, 1.0, 0.01)
for threshold in thresholds:
    y_pred_thresholded = (y_pred_proba >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thresholded).ravel()
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    metrics_df.loc[len(metrics_df)] = [
        threshold, tp, fp, fn, tn,
        recall, precision, accuracy, f1, fpr
    ]

metrics_df['Recall'] = (metrics_df['Recall'] * 100).round(3)
metrics_df['Precision'] = (metrics_df['Precision'] * 100).round(3)
metrics_df['Accuracy'] = (metrics_df['Accuracy'] * 100).round(3)
metrics_df['F1-Score'] = (metrics_df['F1-Score'] * 100).round(3)
metrics_df['FPR'] = (metrics_df['FPR'] * 100).round(3)
    
print("Расчет метрик завершен.")
print(metrics_df)

print("\n" + "="*50)
print("Анализ важности признаков (коэффициенты)")
print("="*50)

coefficients = logreg_pipeline.named_steps['classifier'].coef_[0]

all_feature_names = numerical_features + ohe_feature_names.tolist()

importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Coefficient': coefficients
})

importance_df['Absolute_Coefficient'] = importance_df['Coefficient'].abs()
importance_df = importance_df.sort_values(by='Absolute_Coefficient', ascending=False)
print(importance_df.head(15))

In [ ]:

df = pd.read_csv('Safety Scoring ML - Error matrix Log regression.csv')

df["TPR"] = df["Recall"] / 100
df["FPR"] = df["FPR"] / 100

df = df.sort_values("FPR")
roc_auc = auc(df["FPR"], df["TPR"])

plt.figure(figsize=(8,6))
plt.plot(df["FPR"], df["TPR"], marker='', label=f"ROC (AUC = {roc_auc:.3f})", color='orange')
plt.plot([0,1], [0,1], 'k--', label="Random guess", color='blue')

plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (Recall / TPR)")
plt.title("ROC Curve for Regression Model")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
all_feature_names = numerical_features

importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Coefficient': coefficients
})

importance_df['Absolute_Coefficient'] = importance_df['Coefficient'].abs()
importance_df = importance_df.sort_values(by='Absolute_Coefficient', ascending=False)
importance_df.head(15)

In [ ]:
metrics_df.to_csv('log_regression.csv')

### Feature importance

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Union

def calculate_feature_importance_share(
    importance_scores: Union[np.ndarray, List[float]], 
    feature_names: List[str]
) -> pd.DataFrame:
    """
    Рассчитывает относительный и кумулятивный процент важности признаков.

    Args:
        importance_scores: Массив или список с оценками важности признаков.
        feature_names: Список названий признаков.

    Returns:
        DataFrame с отсортированными признаками и их долей важности.
    """
    if len(importance_scores) != len(feature_names):
        raise ValueError("Размеры массивов 'importance_scores' и 'feature_names' должны совпадать.")
        
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance_Score': np.abs(importance_scores) # Используем абсолютное значение для коэффициентов
    })
    
    # Сортируем по убыванию
    importance_df = importance_df.sort_values(by='Importance_Score', ascending=False)
    
    # Рассчитываем общую сумму важности
    total_importance = importance_df['Importance_Score'].sum()
    
    # Рассчитываем относительный процент
    importance_df['Relative_Share (%)'] = (importance_df['Importance_Score'] / total_importance) * 100
    
    # Рассчитываем кумулятивный процент
    importance_df['Cumulative_Share (%)'] = importance_df['Relative_Share (%)'].cumsum()
    
    return importance_df.reset_index(drop=True)

lgbm_importance = model.feature_importances_
lgbm_feature_names = X_train.columns.tolist()

lgbm_importance_df = calculate_feature_importance_share(lgbm_importance, lgbm_feature_names)

lgbm_importance_df.head(10)




logreg_coeffs = logreg_pipeline.named_steps['classifier'].coef_[0]

all_feature_names = numerical_features

logreg_importance_df = calculate_feature_importance_share(logreg_coeffs, all_feature_names)

print("\n--- Важность признаков для логистической регрессии (доля) ---")
print(logreg_importance_df.head(10))

top_80_percent_features = logreg_importance_df[logreg_importance_df['Cumulative_Share (%)'] <= 80]
print("\n--- 80% самых важных признаков в логистической регрессии ---")
print(top_80_percent_features)

In [ ]:
logreg_importance_df

In [ ]:
lgbm_importance_df[lgbm_importance_df['Cumulative_Share (%)']>80]